# Plantilla Guía para el Trabajo 2: Modelado Supervisado

**Curso:** Introducción a Ciencia de Datos y Machine Learning con Python

---

Este notebook es una plantilla para guiar el desarrollo del Trabajo 2. El objetivo es proporcionar una estructura clara para el proceso de modelado, validación y comparación, asegurando que se cumplan todos los requisitos críticos de la asignación. Reemplazen con sus propios datos y análisis según sea necesario. Tmbn reemplazen los ... con el código o valores que correspondan, sino, no funcionará.

**Pasos sugeridos:**
1.  **Adaptar el código:** Reemplaza los nombres de variables de ejemplo (`'variable_objetivo'`, `'feature_1'`, etc.) con los nombres de las columnas de tu propio dataset. Añade o elimina celdas según las necesidades de tu análisis y modelo.
2.  **Completar el análisis:** Encontrarás celdas de Markdown con la etiqueta `[ESCRIBE AQUÍ]` donde deberás escribir tus interpretaciones, justificaciones y conclusiones.

#Trabajo 2: Análisis de Crédito al Sector Privado y Exportaciones en Piura

Curso: Introducción a Ciencia de Datos y Machine Learning con Python

¿En qué medida el crédito directo del sistema financiero al
sector privado en Piura impacta en el nivel de exportaciones del
departamento?

Objetivo

Evaluar la relación entre el crédito directo otorgado por el sistema financiero al sector privado en Piura y el nivel de exportaciones del departamento.


Grupo: N° 7

Integrantes:

Carlos Fabrizzio Cruz Valencia

Nicolas Alexis Rodriguez Farro

Cristina Sánchez Miranda

Angelica Milagros Velasquez Cabrera

## 1. Carga de Librerías y Datos

In [ ]:
# Librerías para manipulación de datos
import pandas as pd
import numpy as np

# Librerías para visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Librerías para modelado y evaluación
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

# Statsmodels para inferencia
import statsmodels.api as sm

# Configuraciones generales
plt.style.use('seaborn-v0_8-whitegrid') # Fondo blanco y cuadrícula para gráficos seaborn
pd.set_option('display.float_format', lambda x: f'{x:,.4f}') # Formato para floats

In [ ]:
# Carga de datos
# Asegúrate de cargar el dataset limpio y curado del Trabajo 1.
# Reemplaza 'tu_dataset.csv' con el nombre de tu archivo, y read_csv con la función adecuada si es otro formato.
df = pd.read_csv('df_final.csv')

# Muestra un vistazo de los datos (puedes incluir lo mas relevante de tu trabajo1)
df.head()

,fecha,tipo_cambio,credito_total_piura,credito_soles_piura,credito_dolares_piura,exportaciones_piura,credito_total_nacional,exportaciones_nacional,ICEN,ICEN_exp,riesgo_pais_embig
0,2005-01-01,3.2682,930.5421,539.0385,391.5036,208.1825,"38,568.6866","4,131.1994",-0.1700,0.8437,171.8095
1,2005-02-01,3.2588,932.1846,544.4757,387.7089,224.4368,"38,631.6257","3,705.2732",-0.7400,0.4771,171.8097
2,2005-03-01,3.2593,971.2124,566.4315,404.7809,200.1131,"39,155.8295","4,391.8998",-0.8800,0.4148,171.8099
3,2005-04-01,3.2580,988.2234,580.9345,407.2889,178.2377,"40,102.7880","4,097.8322",-0.6800,0.5066,171.8101
4,2005-05-01,3.2549,998.2878,577.6179,420.6699,245.9866,"41,154.8992","4,409.6433",-0.4600,0.6313,171.8102


## 2. Separación de Datos: Training y Test (Paso Crítico)

Este es uno de los pasos más importantes. Reservamos un conjunto de datos (`test`) que **NO USAREMOS** para entrenar, ajustar o comparar modelos. Solo lo usaremos **UNA VEZ** al final para reportar el rendimiento del mejor modelo elegido.

In [ ]:
# --- Define tus variables ---
# Variable dependiente (Y)
TARGET = 'exportaciones_piura'

# Variables independientes (X)
FEATURES = [
    'credito_total_piura',
    'tipo_cambio',
    'ICEN',

    'riesgo_pais_embig',
    'credito_total_nacional',
    'exportaciones_nacional'
]
# Definir X e y
X = df[FEATURES]
y = df[TARGET]

# --- Separación de datos ---
# test_size = 0.2 → 80% entrenamiento, 20% prueba
# random_state = 42 → asegura reproducibilidad del split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Tamaño del conjunto de entrenamiento: {len(X_train)} filas")
print(f"Tamaño del conjunto de prueba: {len(X_test)} filas")


Tamaño del conjunto de entrenamiento: 172 filas
Tamaño del conjunto de prueba: 44 filas


## 3. Modelo Base (Baseline)

Antes de construir modelos complejos, establecemos un "modelo ingenuo" o baseline. Este modelo es muy simple y sirve como punto de referencia. Cualquier modelo que construyamos debe, como mínimo, superar a este baseline.

-   **Para regresión:** El baseline suele predecir el valor medio de la variable objetivo para todas las observaciones.
-   **Para clasificación:** El baseline suele predecir la clase mayoritaria (moda).

In [ ]:

# Baseline para Regresión (predecir la media)
mean_baseline_prediction = y_train.mean()

# Calculamos el MSE del baseline en el conjunto de entrenamiento
baseline_mse = mean_squared_error(y_train, [mean_baseline_prediction] * len(y_train))
#[mean_baseline_prediction] * len(y_train) crea una lista donde todas las predicciones son iguales al promedio, una por cada observación de entrenamiento.
#Si tu modelo es peor que el promedio, algo anda mal.
print(f"Predicción del Baseline (Media de y_train): {mean_baseline_prediction:,.4f}")
print(f"MSE del Baseline en Training: {baseline_mse:,.4f}")

Predicción del Baseline (Media de y_train): 589.0589
MSE del Baseline en Training: 89,605.9318


---
# 4. Ruta de Modelado
A continuación, se presentan las dos rutas posibles para la comparación de modelos. **Elige y desarrolla solo una de ellas.**

### **Opción A: Comparar dos modelos de Regresión (OLS Simple vs. OLS Complejo)**
---

#### 4.A.1. Modelo 1: Regresión Lineal Simple

##### a) Inferencia con `statsmodels`
Usamos `statsmodels` para obtener una tabla de resultados completa que nos permita interpretar los coeficientes, su significancia estadística (p-values) y la bondad de ajuste general del modelo ($R^2$ ajustado). **Este análisis se hace sobre el conjunto de entrenamiento.**

In [ ]:
# Selecciona un subconjunto de variables para el modelo simple
features_simple = ['credito_total_piura', 'tipo_cambio']
X_train_simple = X_train[features_simple]

# Añadimos una constante (intercepto) al modelo
X_train_sm_simple = sm.add_constant(X_train_simple)

# Ajustamos el modelo OLS
model_ols_simple = sm.OLS(y_train, X_train_sm_simple).fit()

# Mostramos el resumen de resultados
print(model_ols_simple.summary())

                             OLS Regression Results                            
Dep. Variable:     exportaciones_piura   R-squared:                       0.539
Model:                             OLS   Adj. R-squared:                  0.533
Method:                  Least Squares   F-statistic:                     98.62
Date:                 Wed, 15 Oct 2025   Prob (F-statistic):           4.14e-29
Time:                         22:39:09   Log-Likelihood:                -1158.2
No. Observations:                  172   AIC:                             2322.
Df Residuals:                      169   BIC:                             2332.
Df Model:                            2                                         
Covariance Type:             nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                 17

**Interpretación de Resultados (Modelo Simple)**

* **Coeficientes:** ¿Qué significan los coeficientes estimados? ¿Son los signos consistentes con la teoría económica o la intuición?

El coeficiente de credito_total_piura es 0.0850 y es consistente con la teoría económica.

* **P-values:** ¿Qué variables son estadísticamente significativas (usualmente p < 0.05)?

La única variable estadísticamente significativa es credito_total_piura (p-value = 0.000)

* **R-cuadrado ajustado:** ¿Qué porcentaje de la variabilidad de la variable objetivo es explicado por el modelo?

El R-cuadrado ajustado del modelo es 0.533, lo que indica que el 53.3 % de la variabilidad observada en la variable objetivo (exportaciones_piura) es explicada por las variables incluidas en el modelo.

* **Diagnósticos:** ¿Hay alguna advertencia importante (ej. multicolinealidad, heterocedasticidad)?

El número de condición (Cond. No. = 5.71e+04) es muy elevado, lo cual advierte sobre posibles problemas de multicolinealidad o diferencias de escala entre las variables independientes

##### b) Predicción y Validación Cruzada con `scikit-learn`
Ahora evaluamos la capacidad predictiva del modelo usando Validación Cruzada (CV). Esto nos da una estimación más robusta de cómo se comportará el modelo en datos no vistos, promediando el rendimiento a través de múltiples divisiones (`k-folds`) del conjunto de entrenamiento.

In [ ]:
# Definimos el modelo de regresión lineal
model_sk_simple = LinearRegression()

# Configuramos la Validación Cruzada (k=5 o k=10 es común)
k = 5 #Elijo el numero de folds
# Calculamos el MSE usando cross_val_score.
# Nota: scikit-learn usa 'neg_mean_squared_error', por lo que invertimos el signo.
mse_scores_simple = -cross_val_score(model_sk_simple, X_train_simple, y_train, cv=k, scoring='neg_mean_squared_error')
#Si usan series de tiempo, usen TimeSeriesSplit en vez!!
#from sklearn.model_selection import TimeSeriesSplit

#tscv = TimeSeriesSplit(n_splits=5)
#mse_scores = -cross_val_score(modelo, X, y, cv=tscv, scoring='neg_mean_squared_error')


print(f"MSEs en cada fold (CV): {mse_scores_simple}")
print(f"MSE Promedio (CV): {np.mean(mse_scores_simple):,.4f} (+/- {np.std(mse_scores_simple):,.4f})")

MSEs en cada fold (CV): [ 29714.65280478  26869.70714796  24533.46669488  37928.84691144
 107621.78720518]
MSE Promedio (CV): 45,333.6922 (+/- 31,471.1329)


**Interpretación de Resultados de CV (Modelo Simple)**

* ¿Cuál es el error de predicción promedio esperado para este modelo? (El MSE promedio).

El error cuadrático medio (MSE) promedio estimado por validación cruzada para el modelo simple es 45,333.6922. Este valor indica cuánto se espera que, en promedio, las predicciones del modelo difieran de los valores reales de las exportaciones de Piura.

* ¿Qué tan estable es el rendimiento del modelo? (Observa la desviación estándar. Un valor alto sugiere que el rendimiento varía mucho dependiendo de los datos de entrenamiento).

La desviación estándar del MSE entre los diferentes folds de la validación cruzada es 31,471.1329. Lo cual sugiere baja estabilidad del modelo y alta sensibilidad a la muestra.


#### 4.A.2. Modelo 2: Regresión Lineal Compleja

Ahora creamos un modelo más complejo. Esto puede significar:
* Añadir más variables.
* Incluir transformaciones (logaritmos, raíces cuadradas).
* Añadir términos polinomiales o interacciones.

In [ ]:
# Usamos todas las variables para el modelo complejo
features_complex = FEATURES # O un subconjunto más grande
X_train_complex = X_train[features_complex]

# OPCIONAL: Añadir términos polinomiales (ej. grado 2)
# poly = PolynomialFeatures(degree=2, include_bias=False)
# X_train_poly = poly.fit_transform(X_train_complex)
# X_train_complex = pd.DataFrame(X_train_poly, columns=poly.get_feature_names_out(features_complex))

# a) Inferencia con statsmodels
# Drop rows with NaN values for statsmodels
X_train_complex_sm = X_train_complex.dropna()
y_train_sm = y_train.loc[X_train_complex_sm.index]

X_train_sm_complex = sm.add_constant(X_train_complex_sm)
model_ols_complex = sm.OLS(y_train_sm, X_train_sm_complex).fit()
print(model_ols_complex.summary())

# b) Predicción con scikit-learn y CV
model_sk_complex = LinearRegression()
mse_scores_complex = -cross_val_score(model_sk_complex, X_train_complex, y_train, cv=k, scoring='neg_mean_squared_error')

print("\n--- Resultados de Validación Cruzada (Modelo Complejo) ---")
print(f"MSEs en cada fold (CV): {mse_scores_complex}")
print(f"MSE Promedio (CV): {np.mean(mse_scores_complex):,.4f} (+/- {np.std(mse_scores_complex):,.4f})")

                             OLS Regression Results                            
Dep. Variable:     exportaciones_piura   R-squared:                       0.691
Model:                             OLS   Adj. R-squared:                  0.679
Method:                  Least Squares   F-statistic:                     61.38
Date:                 Wed, 15 Oct 2025   Prob (F-statistic):           1.58e-39
Time:                         22:39:21   Log-Likelihood:                -1123.8
No. Observations:                  172   AIC:                             2262.
Df Residuals:                      165   BIC:                             2284.
Df Model:                            6                                         
Covariance Type:             nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const             

**Interpretación de Resultados (Modelo Complejo)**

Se incorporaron más variables explicativas:
tipo_cambio, ICEN, riesgo_pais_embig, credito_total_nacional, exportaciones_nacional. Se buscó mejorar la capacidad predictiva del modelo.
Estimación mediante statsmodels y evaluación con scikit-learn.

* **Inferencia:** ¿Cómo cambiaron los coeficientes y su significancia? ¿Mejoró el $R^2$ ajustado?

Al añadir más variables, se observó un cambio en los coeficientes y su
significancia. La variable exportaciones_nacional se vuelve significativa (coef=0.0564, p = 0.000). El R² ajustado mejora de 0.533 a 0.679, mostrando una mayor capacidad explicativa del modelo.

* **Predicción:** ¿El MSE promedio de la validación cruzada mejoró (disminuyó) en comparación con el modelo simple? ¿Aumentó la variabilidad (std dev)?

El MSE promedio disminuye de 45,333.6922 (modelo simple) a 32,746.4403,
con una desviación estándar menor (18,667.8817). Esto refleja un modelo más estable, preciso y robusto.



#### 4.A.3. Comparación de Modelos de Regresión

In [ ]:
# Creamos un DataFrame para comparar los resultados de CV
comparison_data = {
    'Modelo': ['Baseline', 'OLS Simple', 'OLS Complejo'],
    'MSE Promedio (CV)': [baseline_mse, np.mean(mse_scores_simple), np.mean(mse_scores_complex)],
    'Std Dev (CV)': [0, np.std(mse_scores_simple), np.std(mse_scores_complex)]
}

comparison_df = pd.DataFrame(comparison_data)
comparison_df.set_index('Modelo', inplace=True)

print("--- Tabla Comparativa de Rendimiento Predictivo ---")
display(comparison_df.style.highlight_min(subset='MSE Promedio (CV)', color='lightgreen'))
#Resaltando en color verde claro el valor más bajo de la columna 'MSE Promedio (CV)'

--- Tabla Comparativa de Rendimiento Predictivo ---


,MSE Promedio (CV),Std Dev (CV)
Modelo,,
Baseline,89605.931807,0.000000
OLS Simple,45333.692153,31471.132899
OLS Complejo,32746.440318,18667.881661


**Decisión del Mejor Modelo (Ruta A)**

* Basado en el MSE promedio de la validación cruzada, ¿qué modelo tiene el mejor rendimiento predictivo?

El modelo OLS Complejo presenta el menor error promedio y mayor estabilidad.

* Considera también la complejidad del modelo. Si la mejora en el MSE es muy pequeña, ¿justifica añadir más variables y complejidad (principio de parsimonia)?

El modelo mantiene una capacidad predictiva sólida y no presenta sobreajuste.

La estimación de validación cruzada fue robusta y generaliza bien a nuevos datos.

---
## 5. Evaluación Final sobre el Conjunto de Prueba (Test Set)

**¡Llegó el momento de la verdad!**

Ahora tomamos el **único modelo ganador** que elegimos en la sección anterior, lo re-entrenamos con **TODO** el conjunto de entrenamiento (`X_train`, `y_train`) y lo evaluamos en el conjunto de prueba (`X_test`, `y_test`) que habíamos guardado.

In [ ]:
# --- REEMPLAZA CON TU MODELO GANADOR ---
# Ejemplo si el ganador fue el 'OLS Complejo'
best_model = LinearRegression()
X_train_final = X_train[features_complex]
X_test_final = X_test[features_complex]

# 1. Re-entrenar el modelo con TODOS los datos de entrenamiento
best_model.fit(X_train_final, y_train)

# 2. Hacer predicciones en el conjunto de prueba
final_predictions = best_model.predict(X_test_final)

# 3. Evaluar el rendimiento final
# Si tu mejor modelo fue de REGRESIÓN:
final_mse = mean_squared_error(y_test, final_predictions)
final_r2 = r2_score(y_test, final_predictions)

print("--- Evaluación Final del Modelo de Regresión ---")
print(f"MSE en Test: {final_mse:,.4f}")
print(f"R2 en Test: {final_r2:,.4f}")

# Si tu mejor modelo fue de CLASIFICACIÓN (ej. Logit):
# best_model_class = LogisticRegression(random_state=42, max_iter=1000)
# best_model_class.fit(X_train_c, y_train_c)
# final_predictions_c = best_model_class.predict(X_test_c)
# print("\n--- Evaluación Final del Modelo de Clasificación ---")
# print(classification_report(y_test_c, final_predictions_c))

--- Evaluación Final del Modelo de Regresión ---
MSE en Test: 20,189.4841
R2 en Test: 0.7002


**Análisis de Resultados Finales**



* Compara el rendimiento en el conjunto de prueba con el rendimiento promedio obtenido en la Validación Cruzada. ¿Son similares?
    * Si el rendimiento en `test` es mucho peor, podría ser una señal de sobreajuste (overfitting).
    * Si es similar, confirma que tu estimación de CV era robusta.
* Este es el rendimiento final esperado de tu modelo en nuevos datos.

## 6. Conclusiones

Modelo ganador: El modelo OLS Complejo, con mejor desempeño (MSE = 20,189; R²
= 0.70).

Hallazgos clave: La variable exportaciones_nacional fue el predictor más relevante y significativo.

Limitaciones: Posible multicolinealidad y exclusión de variables relevantes.

Próximos pasos: Probar modelos polinomiales, ARIMA o métodos de Machine Learning
para mejorar la predicción.
